# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [129]:
import pandas as pd
import numpy as np
import folium


In [71]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: True
fatal: destination path 'DataScienceSS20' already exists and is not an empty directory.


In [ ]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv(path+'/DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [72]:
# install 
!pip install scikit-optimize

### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [73]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [140]:
# simple model only using Random Forest

X, y = load_iris(True)
X= np.array(X)
y= np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.75,random_state=0)


X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)


In [141]:
opt = BayesSearchCV(
    SVC(),
    {
        'C': Real(1e-6, 1e+6, prior='log-uniform'),
        'gamma': Real(1e-6, 1e+1, prior='log-uniform'),
        'degree': Integer(1,8),
        'kernel': Categorical(['linear', 'poly', 'rbf']),
     },
     n_iter=32,
     random_state=0
     )

opt.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=None, error_score='raise',
              estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                            class_weight=None, coef0=0.0,
                            decision_function_shape='ovr', degree=3,
                            gamma='scale', kernel='rbf', max_iter=-1,
                            probability=False, random_state=None,
                            shrinking=True, tol=0.001, verbose=False),
              fit_params=None, iid=True, n_iter=32, n_jobs=1, n_points=1,
              optimizer_kwargs=No...
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': Real(low=1e-06, high=1000000.0, prior='log-uniform', transform='identity'),
                             'degree': Integer(low=1, high=8, prior='uniform', transform='identity'),
                             'gamma': Real(low=1e-06, high=10.0, prior='log-uniform', transform='identity'),
                             'kernel': Categorical(cate

In [142]:
opt.best_score_

0.9821428571428571

In [143]:
opt.best_params_

OrderedDict([('C', 1.3361910455737007),
             ('degree', 5),
             ('gamma', 0.11283439533114079),
             ('kernel', 'linear')])

In [144]:
pipe = Pipeline([
    ('model', SVR()) #just put one model as placeholder
])

In [145]:
# Linear Regression

from sklearn.linear_model import LinearRegression
lin_regr = {
  'model': Categorical([LinearRegression()]),
}


In [146]:
# Random Forest Regression

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
#X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
rf_regr = {
    'model': Categorical([RandomForestRegressor()]),
    'model__n_estimators': Integer(10, 100, 'log-uniform'),
}

In [147]:
# SVM Regression

from sklearn.preprocessing import StandardScaler
n_samples, n_features = 10, 5
rng = np.random.RandomState(0)
y_train = rng.randn(n_samples)
X_train = rng.randn(n_samples, n_features)
svm_regr = {
    'model':Categorical([SVR()]),
    'model__C': Real(1e-6, 1e+6, prior='log-uniform'),
    'model__gamma': Real(1e-6, 1e+1, prior='log-uniform'),
}

In [148]:
# model comparison
opt = BayesSearchCV(
    pipe,
    [(svm_regr, 10), (rf_regr, 10), (lin_regr,1)], # (parameter space, # of evaluations)
    cv=5,
     n_iter=32,
    random_state=0,
    n_jobs=4, #parallelize
    scoring='neg_mean_squared_error'
)

In [149]:
opt.fit(X_train,y_train)

BayesSearchCV(cv=5, error_score='raise',
              estimator=Pipeline(memory=None,
                                 steps=[('model',
                                         SVR(C=1.0, cache_size=200, coef0=0.0,
                                             degree=3, epsilon=0.1,
                                             gamma='scale', kernel='rbf',
                                             max_iter=-1, shrinking=True,
                                             tol=0.001, verbose=False))],
                                 verbose=False),
              fit_params=None, iid=True, n_iter=32, n_jobs=4, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_stat...
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False),), prior=None),
                               'model__n_estimators': Integer(low

In [150]:
opt.best_score_


-1.0980889680984682

In [155]:
opt.best_params_

OrderedDict([('model',
              RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                                    max_samples=None, min_impurity_decrease=0.0,
                                    min_impurity_split=None, min_samples_leaf=1,
                                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                                    n_estimators=11, n_jobs=None, oob_score=False,
                                    random_state=None, verbose=0, warm_start=False)),
             ('model__n_estimators', 11)])